# 랭체인(LangChain) SQL Agents 예제
## 작성자 : AISchool ( http://aischool.ai/%ec%98%a8%eb%9d%bc%ec%9d%b8-%ea%b0%95%ec%9d%98-%ec%b9%b4%ed%85%8c%ea%b3%a0%eb%a6%ac/ )
## Reference : https://python.langchain.com/docs/use_cases/sql/agents

![](https://python.langchain.com/assets/images/sql_usecase-d432701261f05ab69b38576093718cf3.png)

# Sample SQL DB 다운로드

## Reference : https://www.sqlitetutorial.net/sqlite-sample-database/

![](https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)

In [ ]:
!wget https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip -O chinook.zip

--2024-03-10 09:33:16--  https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
Resolving www.sqlitetutorial.net (www.sqlitetutorial.net)... 172.67.172.250, 104.21.30.141, 2606:4700:3037::ac43:acfa, ...
Connecting to www.sqlitetutorial.net (www.sqlitetutorial.net)|172.67.172.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 305596 (298K) [application/zip]
Saving to: ‘chinook.zip’

chinook.zip         100%[===================>] 298.43K  --.-KB/s    in 0.02s   

2024-03-10 09:33:16 (13.9 MB/s) - ‘chinook.zip’ saved [305596/305596]



In [ ]:
!unzip chinook.zip

Archive:  chinook.zip
replace chinook.db? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: chinook.db              


# LangChain 라이브러리 설치

In [ ]:
!pip install --upgrade --quiet langchain langchain-community langchain-openai chromadb

# OpenAI API Key 설정

In [ ]:
OPENAI_KEY = "여러분의_OPENAI_API_KEY"

# chinook.db 불러오기

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM artists LIMIT 10;")

sqlite
['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [ ]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_KEY)

# Using a dynamic few-shot prompt

* 에이전트 성능을 최적화하기 위해, 우리는 **도메인 특화 지식(domian-specific
knowledge)**을 포함한 맞춤형 프롬프트를 제공할 수 있습니다.
* 이 경우, **사용자 입력에 기반하여 동적으로 few-shot 프롬프트를 구축하는 예제 선택기**와 함께 few-shot prompt를 생성할 것입니다.
* 이것은 **모델이 참조로 사용할 수 있는 관련 쿼리를 프롬프트에 삽입함으로써 모델이 더 나은 쿼리를 만들 수 있도록 도울 것**입니다.

In [ ]:
examples = [
    {   "input": "List all artists.",
        "query": "SELECT * FROM artists;"},
    {
        "input": "Find all albums for the artist 'AC/DC'.",
        "query": "SELECT * FROM albums WHERE ArtistId = (SELECT ArtistId FROM artists WHERE Name = 'AC/DC');",
    },
    {
        "input": "List all tracks in the 'Rock' genre.",
        "query": "SELECT * FROM tracks WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock');",
    },
    {
        "input": "Find the total duration of all tracks.",
        "query": "SELECT SUM(Milliseconds) FROM tracks;",
    },
    {
        "input": "List all customers from Canada.",
        "query": "SELECT * FROM customers WHERE Country = 'Canada';",
    },
    {
        "input": "How many tracks are there in the album with ID 5?",
        "query": "SELECT COUNT(*) FROM tracks WHERE AlbumId = 5;",
    },
    {
        "input": "Find the total number of invoices.",
        "query": "SELECT COUNT(*) FROM invoices;",
    },
    {
        "input": "List all tracks that are longer than 5 minutes.",
        "query": "SELECT * FROM tracks WHERE Milliseconds > 300000;",
    },
    {
        "input": "Who are the top 5 customers by total purchase?",
        "query": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM invocies GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;",
    },
    {
        "input": "Which albums are from the year 2000?",
        "query": "SELECT * FROM albums WHERE strftime('%Y', ReleaseDate) = '2000';",
    },
    {
        "input": "How many employees are there",
        "query": 'SELECT COUNT(*) FROM "employees"',
    },
]

* 이제 **예제 선택기(example selector)**를 생성할 수 있습니다.
* 이것은 실제 사용자 입력을 받아 우리의 few-shot prompt에 추가할 일부 예제들을 선택할 것입니다.
* 우리는 **SemanticSimilarityExampleSelector**를 사용할 것인데, 이것은 우리가 설정한 임베딩과 벡터 스토어를 사용하여 의미 검색을 수행하여 입력과 가장 유사한 예제들을 찾을 것입니다.

In [ ]:
from langchain.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(openai_api_key=OPENAI_KEY),
    Chroma,
    k=5,
    input_keys=["input"],
)

* 이제 예제 선택기, 각 예제를 형식화하기 위한 예제 프롬프트, 그리고 형식화된 예제들 앞뒤에 붙일 문자열 접두사와 접미사를 사용하여 FewShotPromptTemplate를 생성할 수 있습니다.

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

system_prefix = """당신은 SQL 데이터베이스와 상호 작용하도록 설계된 에이전트입니다.
입력된 질문을 기반으로, {dialect} 쿼리 문법에 맞는 쿼리를 생성하고 실행한 다음 쿼리 결과를 확인하여 답변을 반환하세요.
사용자가 특정한 예제 수를 요청하지 않는 한, 항상 쿼리 결과를 최대 {top_k}개로 제한하세요.
데이터베이스 내에서 가장 흥미로운 예제를 반환하기 위해 관련 컬럼으로 결과를 정렬할 수 있습니다.
특정 테이블의 모든 컬럼을 조회하는 쿼리는 실행하지 마세요, 질문에 주어진 관련 컬럼만 요청하세요.
데이터베이스와 상호 작용하기 위한 도구에 접근할 수 있습니다.
주어진 도구만 사용하세요. 도구를 통해 반환된 정보만을 사용하여 최종 답변을 구성하세요.
쿼리를 실행하기 전에 반드시 쿼리를 다시 확인하세요. 쿼리 실행 중 오류가 발생한 경우, 쿼리를 다시 작성하고 시도하세요.

데이터베이스에 DML 문장(INSERT, UPDATE, DELETE, DROP 등)을 실행하지 마세요.

질문이 데이터베이스와 관련 없어 보이면, 단순히 "모르겠습니다"라고 답변하세요.

다음은 사용자 입력과 해당하는 SQL 쿼리의 예시입니다:"""

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=system_prefix,
    suffix="",
)

In [ ]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [ ]:
# Example formatted prompt
prompt_val = full_prompt.invoke(
    {
        "input": "How many arists are there",
        "top_k": 5,
        "dialect": "SQLite",
        "agent_scratchpad": [],
    }
)
print(prompt_val.to_string())

System: 당신은 SQL 데이터베이스와 상호 작용하도록 설계된 에이전트입니다.
입력된 질문을 기반으로, SQLite 쿼리 문법에 맞는 쿼리를 생성하고 실행한 다음 쿼리 결과를 확인하여 답변을 반환하세요.
사용자가 특정한 예제 수를 요청하지 않는 한, 항상 쿼리 결과를 최대 5개로 제한하세요.
데이터베이스 내에서 가장 흥미로운 예제를 반환하기 위해 관련 컬럼으로 결과를 정렬할 수 있습니다.
특정 테이블의 모든 컬럼을 조회하는 쿼리는 실행하지 마세요, 질문에 주어진 관련 컬럼만 요청하세요.
데이터베이스와 상호 작용하기 위한 도구에 접근할 수 있습니다.
주어진 도구만 사용하세요. 도구를 통해 반환된 정보만을 사용하여 최종 답변을 구성하세요.
쿼리를 실행하기 전에 반드시 쿼리를 다시 확인하세요. 쿼리 실행 중 오류가 발생한 경우, 쿼리를 다시 작성하고 시도하세요.

데이터베이스에 DML 문장(INSERT, UPDATE, DELETE, DROP 등)을 실행하지 마세요.

질문이 데이터베이스와 관련 없어 보이면, 단순히 "모르겠습니다"라고 답변하세요.

다음은 사용자 입력과 해당하는 SQL 쿼리의 예시입니다:

User input: List all artists.
SQL query: SELECT * FROM artists;

User input: How many employees are there
SQL query: SELECT COUNT(*) FROM "employees"

User input: How many tracks are there in the album with ID 5?
SQL query: SELECT COUNT(*) FROM tracks WHERE AlbumId = 5;

User input: List all tracks in the 'Rock' genre.
SQL query: SELECT * FROM tracks WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock');

User input:

In [ ]:
agent = create_sql_agent(
    llm=llm,
    db=db,
    prompt=full_prompt,
    verbose=True,
    agent_type="openai-tools",
)

In [ ]:
agent.invoke({"input": "artists는 몇명이 있나요?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `SELECT COUNT(*) FROM artists`


[(275,)]현재 데이터베이스에는 275명의 아티스트가 있습니다.

> Finished chain.


{'input': 'artists는 몇명이 있나요?', 'output': '현재 데이터베이스에는 275명의 아티스트가 있습니다.'}

In [ ]:
test_agent = create_sql_agent(
    llm=llm,
    db=db,
    verbose=True,
    agent_type="openai-tools",
)

In [ ]:
test_agent.invoke({"input": "artists는 몇명이 있나요?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with ``


albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracks
Invoking: `sql_db_schema` with `{'table_names': 'artists'}`



CREATE TABLE artists (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from artists table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/
Invoking: `sql_db_query` with `SELECT COUNT(*) AS artist_count FROM artists`


[(275,)]There are 275 artists in the database.

> Finished chain.


{'input': 'artists는 몇명이 있나요?',
 'output': 'There are 275 artists in the database.'}

# Dealing with high-cardinality columns

* 주소, 노래 이름 또는 아티스트와 같은 고유명사를 포함한 컬럼을 필터링하기 위해서는, 데이터를 올바르게 필터링하기 위해 먼저 철자를 이중으로 확인해야 합니다.

* 이를 달성하기 위해 데이터베이스에 존재하는 모든 고유명사의 고유 값들로 벡터 저장소를 생성할 수 있습니다. 그런 다음 에이전트가 사용자의 질문에 고유명사를 포함할 때마다 그 벡터 저장소를 쿼리하여 해당 단어의 정확한 철자를 찾을 수 있습니다. 이 방식으로, 에이전트는 타겟 쿼리를 구축하기 전에 사용자가 어떤 엔티티를 참조하는지 확실히 이해할 수 있습니다.

* 먼저, 우리가 원하는 각 엔티티의 고유 값들을 필요로 하는데, 이를 위해 결과를 요소 목록으로 파싱하는 함수를 정의해야 합니다.

In [ ]:
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=OPENAI_KEY)

In [ ]:
import ast
import re

# 고유한 값들을 리스트로 반환
def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


artists = query_as_list(db, "SELECT Name FROM artists")
albums = query_as_list(db, "SELECT Title FROM albums")

In [ ]:
albums

['Houses Of The Holy',
 'Tangents',
 'Zooropa',
 'The Doors',
 'Vinicius De Moraes',
 'Use Your Illusion I',
 'Temple of the Dog',
 '20th Century Masters - The Millennium Collection: The Best of Scorpions',
 'Beethoven: Symhonies Nos.  &',
 'How To Dismantle An Atomic Bomb',
 'Pearl Jam',
 'Live At Donington  (Disc )',
 'Bach: Goldberg Variations',
 'The Song Remains The Same (Disc )',
 'Original Soundtracks',
 'A Copland Celebration, Vol. I',
 'Prokofiev: Symphony No.',
 'Master Of Puppets',
 'Live [Disc ]',
 'Fireball',
 'Roda De Funk',
 'Djavan Ao Vivo - Vol.',
 'The Colour And The Shape',
 'Living After Midnight',
 'Milton Nascimento Ao Vivo',
 'Piece Of Mind',
 'Big Ones',
 'Presence',
 'Back to Black',
 "Pavarotti's Opera Made Easy",
 'Unplugged [Live]',
 'Live After Death',
 'Greatest Hits II',
 'Lost, Season',
 'The Singles',
 'Quanta Gente Veio ver--Bônus De Carnaval',
 'Cesta Básica',
 'In Your Honor [Disc ]',
 'Cidade Negra - Hits',
 'Handel: Music for the Royal Fireworks (O

In [ ]:
artists

['R.E.M. Feat. KRS-One',
 'Frank Zappa & Captain Beefheart',
 'Göteborgs Symfoniker & Neeme Järvi',
 'Jorge Ben',
 'Frank Sinatra',
 'Gustavo & Andres Veiga & Salazar',
 'The Posies',
 'AC/DC',
 'London Symphony Orchestra & Sir Charles Mackerras',
 'Marillion',
 'Emerson String Quartet',
 'Creedence Clearwater Revival',
 'Berliner Philharmoniker & Hans Rosbaud',
 'JET',
 'Joe Satriani',
 'Christina Aguilera featuring BigElf',
 'Miles Davis',
 'Ton Koopman',
 'Whitesnake',
 'Barão Vermelho',
 'Pink Floyd',
 'Chico Science & Nação Zumbi',
 'Body Count',
 "Paul D'Ianno",
 'Os Mutantes',
 'Apocalyptica',
 'Regina Spektor',
 'Xis',
 'Sir Georg Solti, Sumi Jo & Wiener Philharmoniker',
 'Def Leppard',
 'The Doors',
 'C. Monteverdi, Nigel Rogers - Chiaroscuro; London Baroque; London Cornett & Sackbu',
 'Buddy Guy',
 'Dread Zeppelin',
 'Alberto Turco & Nova Schola Gregoriana',
 'Nação Zumbi',
 'Nash Ensemble',
 'Nega Gizza',
 'Billy Cobham',
 'Caetano Veloso',
 'Os Cariocas',
 'Peter Tosh',
 'C

In [ ]:
from langchain.agents.agent_toolkits import create_retriever_tool

vector_db = Chroma.from_texts(artists + albums, OpenAIEmbeddings(openai_api_key=OPENAI_KEY))
retriever = vector_db.as_retriever(search_kwargs={"k": 10})
description = """필터링할 값들을 조회하는 데 사용합니다. 입력은 고유명사의 대략적인 철자이며, 출력은 유효한 고유명사입니다. 검색과 가장 유사한 명사를 사용하세요."""
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

In [ ]:
system = """당신은 SQL 데이터베이스와 상호 작용하도록 설계된 에이전트입니다.
입력된 질문에 대해, 구문적으로 올바른 {dialect} 쿼리를 생성하여 실행한 다음, 쿼리 결과를 확인하고 답변을 반환하세요.
사용자가 얻고자 하는 예제의 특정 수를 지정하지 않는 한, 항상 쿼리 결과를 최대 {top_k}개로 제한하세요.
데이터베이스에서 가장 흥미로운 예제를 반환하기 위해 관련 컬럼으로 결과를 정렬할 수 있습니다.
특정 테이블의 모든 컬럼을 조회하는 쿼리는 실행하지 마세요. 질문에 주어진 관련 컬럼만 요청하세요.
데이터베이스와 상호 작용하기 위한 도구에 접근할 수 있습니다.
주어진 도구만 사용하세요. 도구를 통해 반환된 정보만을 사용하여 최종 답변을 구성하세요.
쿼리를 실행하기 전에 반드시 쿼리를 다시 확인하세요. 쿼리 실행 중 오류가 발생한다면, 쿼리를 다시 작성하고 시도하세요.

데이터베이스에 어떠한 DML 문장도 (INSERT, UPDATE, DELETE, DROP 등) 실행하지 마세요.

고유명사를 필터링해야 하는 경우, 반드시 "search_proper_nouns" 도구를 사용하여 필터 값에 대한 조회를 먼저 해야 합니다!

다음 테이블에 접근할 수 있습니다: {table_names}

질문이 데이터베이스와 관련이 없어 보이면, 단순히 "모르겠습니다"라고 답변하세요."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{input}"), MessagesPlaceholder("agent_scratchpad")]
)
agent = create_sql_agent(
    llm=llm,
    db=db,
    extra_tools=[retriever_tool],
    prompt=prompt,
    agent_type="openai-tools",
    verbose=True,
)

In [ ]:
agent.invoke({"input": "alis in chain 앨범은 몇개를 가지고 있나요?"})



> Entering new SQL Agent Executor chain...

Invoking: `search_proper_nouns` with `{'query': 'alis in chain'}`


Alice In Chains

Aisha Duo

A-Sides

Azymuth

Alanis Morissette

One By One

Ace Of Spades

Up An' Atom

Walking Into Clarksdale

In Step
Invoking: `sql_db_query` with `SELECT COUNT(*) FROM albums WHERE ArtistId IN (SELECT ArtistId FROM artists WHERE Name = 'Alice In Chains')`


[(1,)]Alice In Chains는 1개의 앨범을 가지고 있습니다.

> Finished chain.


{'input': 'alis in chain 앨범은 몇개를 가지고 있나요?',
 'output': 'Alice In Chains는 1개의 앨범을 가지고 있습니다.'}